In [12]:
import pandas as pd
from sklearn import linear_model

<b><h3>Categorical Variables: Counting Eggs in the Age of Robotic Chickens</h3></b>

A categorical variable, as the name suggests, is used to represent categories or labels. For instance, a categorical variable could represent major cities in the world, the four seasons in a year, the industry of a company etc. The number of category values is always finite in a real-worlld dataset. The values may be represented numerically. However, unlike other numerical variables, the values of a categorical variable cannot be ordered with respect to one another. They are called nonordinal. 

The vocabulary of a document corpus can be interpreted as a large categorical variable, with the categories being unique words.  It can be computationally expensive to represent so many distinct categories. If a category appears multiple times in a data point (document), then we can represent it as a count, and represent all of the categories through their count statistics. This is called bin counting.

<b><h3>Encoding Categorical Variables</h3></b>

The categories of a categorical variable are usually not numeric. Eye color can be 'black', 'blue', 'brown' etc. Thus an encoding method is needed to turn these nonnumeric categories into numbers. 

<b><h4>One-Hot Encoding</h4></b>

A better method is to use a group of bits. Each bit represent a possible category. If the variable cannot belong to multiple categories at once, then only one bit in the group can be 'on'. This is called one-hot encoding, and it is implemented i scikit-learn as sklearn.preprocessing.OneHotEncoder. Each of the bits is a feature. Thus a categorical variable with k possible categories is encoded as a feature vector of length k.

One hot encoding uses one more bit than is strictly necessary. If we see that k-1 of the bits are 0, then the last bit must be 1 because the variable must take on one of the k values .( Mathematically, one can write this constraint as "the sum of all bits must be equal to 1".) Thus, we have a linear dependency on our hands, Linear dependent features mean that the trained linear models will not be unique. Different linear combinations of the features can make the same predictions so it would be difficult explaining the effect of a feature on the prediction.

<b><h4>Dummy Coding</h4></b>

The problem with one-hot encoding is that it allows for k degrees of freedom, while the variable itself needs only k-1. Dummy coding removes the extra degree of freedom by using only k-1 features in the representation. One feature is thrown under the buss and represented by the vector of all zeros. This is known as the reference category. Dummy coding and one-hot encoding are both implemented in pandas as pandas.get_dummies (to get dummy coding specify drop_first=True). The outcome of modeling with dummy coding is more interpretable than with one-hot encoding.

In [13]:
# Define a toy dataset of apartment rental prices in New York, San Francisco, and Seattle

df = pd.DataFrame({
    'City' : ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 'Seattle', 'Seattle', 'Seattle'],
    'Rent' : [3999,4000,4001,3499,3500,3501,2499,2500,2501]
})

df

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [14]:
df['Rent'].mean()

3333.3333333333335

In [15]:
# Convert the categorical variables in the DataFrame to one-hot encoding and fit a linear regression model
one_hot_df = pd.get_dummies(df, prefix=['city'])

In [16]:
one_hot_df

,Rent,city_NYC,city_SF,city_Seattle
0,3999,False,True,False
1,4000,False,True,False
2,4001,False,True,False
3,3499,True,False,False
4,3500,True,False,False
5,3501,True,False,False
6,2499,False,False,True
7,2500,False,False,True
8,2501,False,False,True


In [18]:
model = linear_model.LinearRegression()

In [19]:
model.fit(one_hot_df[['city_NYC', 'city_SF', 'city_Seattle']], one_hot_df['Rent'])

LinearRegression()

In [21]:
model.coef_

array([ 166.66666667,  666.66666667, -833.33333333])

In [22]:
model.intercept_

3333.3333333333335

In [23]:
# Train a linear regression model on dummy code
# Specify the 'drop_first' flag to get dummy coding
dummy_df = pd.get_dummies(df, prefix=['city'], drop_first=True)
dummy_df

,Rent,city_SF,city_Seattle
0,3999,True,False
1,4000,True,False
2,4001,True,False
3,3499,False,False
4,3500,False,False
5,3501,False,False
6,2499,False,True
7,2500,False,True
8,2501,False,True


In [24]:
model.fit(dummy_df[['city_SF', 'city_Seattle']], dummy_df['Rent'])
model.coef_

array([  500., -1000.])

In [25]:
model.intercept_

3500.0

With one-hot encoding, the intercept term represents the global mean of the target variable. Rent and each of the linear coefficients represent how much that city's average rent differs from the global mean. 

With the dummy coding, the bias coefficient represents the mean value of the response variable y for the reference category. The coefficient for the ith feature is equal to the difference between the mean response value for the ith category and the mean of the reference category.

<b><h4>Effect Coding</h4></b>

Another variant of categorical variable encoding is effect coding. Effect codiing is very similar to dummy coding with the difference that the reference category is now represented by the vector of all -1's. Effect coding is very similar to dummy coding, but results in linear regression models that are even simpler to interpret. The intercept term represents the global mean of the target variable, the individual coefficients indicate how much the means of the individual categories differ from the global mean. (This called the main effect of the category or level, hence the name 'effect coding'). 

Comparing the model built by effect coding to previous results, you realize that one-hot encoding actually come up with
the same intercept and coefficients, but in that case there are linear coefficients for each city. In effect coding, no single feature represents the reference category, so the effect of the reference category needs to be separately computed as the negative sum of the coefficients of all other categories.


In [27]:
effect_df = dummy_df.copy()
effect_df.loc[3:5, ['city_SF', 'city_Seattle']] = -1.0
effect_df

,Rent,city_SF,city_Seattle
0,3999,True,False
1,4000,True,False
2,4001,True,False
3,3499,-1.0,-1.0
4,3500,-1.0,-1.0
5,3501,-1.0,-1.0
6,2499,False,True
7,2500,False,True
8,2501,False,True


In [28]:
model.fit(effect_df[['city_SF', 'city_Seattle']], effect_df['Rent'])

LinearRegression()

In [29]:
model.coef_

array([ 666.66666667, -833.33333333])

In [30]:
model.intercept_

3333.3333333333335

<b><h4>Pros and Cons of Categorical Variable Encodings</h4></b>

One-hot, dummy, and effect coding are very similar to one another. They each have pros and cons. One-hot encoding is redundant, which allows for multiple valid models for the same problem. The nonuniqueness is sometimes problematic for interpretation, but the advantage is that each feature corresponds to a category. Moreover, missing data can be encoded as the all-zero vector, and the output should be the overall mean of the target variable.

Dummy coding and effect coding are not redundant. They give rise to unique and interpretable models. The downside of dummy coding is that it cannot easily handle missing data, since the all-zeros vector is already mapped to the reference category. It also encodes the effect of each category relative to the reference category, which may look strange.

Effect coding avoids this problem by using a different code for the reference category, but the vector of all -1's is a dense vector, which is expensive for both storage and computation. For this reason, popular ML software packages such as pandas and scikit-learn have opted for dummy coding or one-hot encoding instead of effect coding.

All three encoding techniques break down when the number of categories becomes very large. Different strategies are needed to handle extremely large categorical variables.

<b><h3>Dealing with Large Categorical Variables</h3></b>

Automated data collection on the internet can generate large categorical variables. This is common in applications such as targeted advertising and fraud detection. 

In targeted advertising, the task is to match a user with a set of ads. Features include the user ID, the website domain for the ad, the search query, the current page, and all possible pairwise conjunctions of those features. Each of these is a very large categorical variable. The challenge is to find a good feature representation that is memory efficient, yet produces accurate models that are fast to train. Existing solutions are as follows:
- Do nothing fancing with the encoding. Use a simple model that is cheap to train. Feed one-hot encoding into a linear model.
- Compress the features. There are two choices;
a. Feature hashing, popular with linear models
b. Bin counting, popular with linear models as well as trees



<b><h4>Feature Hashing</h4></b>

A hash function is a deterministic function that maps a potentially unbounded integer to a finite integer range [1,m]. Since the input domain is potentially larger than the output range, multiple numbers may get mapped to the same output. This is called a collision. A uniform hash function ensures that roughly the same number of numbers are mapped into each of the m bins.

We can think of a hash function as a machine that intakes numbered balls (keys) and routes them to one of m bins. Balls with the same number will always get routed to the same bin. This maintains the feature space while reducing the storage and processing time during machine learning training and evaluation cycles.

Hash functions can be constructed for any object that can be represented numerically (which is true for any data that can be stored on a computer).

When there are many features, storing the feature vector could take up a lot of space. Feature hashing compresses the original feature vector into an m-dimensional vector by applying a hash function to the feature ID. For instance, if the original features were words in a document, then the hashed version would have a fixed vocubulary size of m, no matter how many uniqu words were in the input.

In [ ]:
# Feature hashing for word features
def hash_features(word_list, m):
    output = [0]*m
    for word in word_list:
        index = hash(word) % m
        output[index] += 1
    return output

Another variation of feature hashing adds a sign component, so that counts are either added to or subtracted from the hashed bin. Statistically speaking, this ensures that the inner products beween hashed features are equal in expectation to those of the original features. The value of the inner product after hashing is within O(1/√m) of the original inner product, so the size of the hash table m can be selected based on acceptable errors. In practice, picking the right m could take some trial and error.

Feature hashing can be used for models that involve the inner product of feature vectors and coefficients, such as linear models and kernel methods. It has been demonstrated to be successful in the task of spam filtering. One downside of feature hashing is that the hashed features, being aggregates of original features, are no longer interpretable.

Feature hashing benefits us computationally, sacrificing immediate user interpretability. This is an easy trade-off to accept when progressing from data exploration and visualization into a machine learning pipeline for large datasets.

In [33]:
# Signed feature hashing
def hash_features(word_list, m):
    output = [0]*m
    for word in word_list:
        index = hash(word) % m
        sign_bit = hash(word) % 2
        if (sign_bit == 0):
            output[index] -= 1
        else:
            output[index] += 1
    return output

In [34]:
# Feature hashing (aka the hashing trick)
review_df = pd.read_csv('../datasets/yelp.csv')

In [38]:
m = len(review_df['business_id'].unique())  # 4174 unique values in business_id categorical column
m

4174

In [66]:
from sklearn.feature_extraction import FeatureHasher

output = []
for value in review_df.business_id.values:
    output.append([value])

h = FeatureHasher(n_features=m, input_type='string')

f = h.transform(output) # expects a list of a list of strings

In [67]:
review_df['business_id'].unique().tolist()[0:5]

['9yKzy9PApeiPPOUJEtnvkg',
 'ZRJwVLyzEJq1VAihDhYiow',
 '6oRAC4uyJCsJl1X0WZpVSA',
 '_1QQZuf4zZOyFCvXc0o6Vg',
 '6ozycU1RpktNG2-1BroVtw']

In [68]:
f.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [79]:
from sys import getsizeof
print('Our pandas Series, in bytes', getsizeof(review_df['business_id']))
print('Our hashed numpy array, in bytes', getsizeof(f))
print('Our one-hot encoded numpy array, in bytes', getsizeof(
                pd.get_dummies(review_df['business_id'])))

Our pandas Series, in bytes 790144
Our hashed numpy array, in bytes 48
Our one-hot encoded numpy array, in bytes 41740144


In [70]:
f.shape

(10000, 4174)

<b><h4>Bin Counting</h4></b>

Bin counting is one of the perennial rediscoveries in machine learning. It has been reinvented and used in a variety of applications, from ad clicking through rate prediction to hardware branch prediction. The idea of bin counting is simple : rather than using the value of the categorical value as the feature, instead use use the conditional probability of the target under that value. In other words, instead of encoding the identity of the categorical value, we compute the association statistics between that value and the target that we wish to predict. Under Bayes classifiers, this statistic is the conditional probability of the class under the assumption that all features are independent.

Bin counting assumes that historical data is available for computing the statistics. For example say we have click data for users, then because on the number of times a user has clicked on any add and the number of times the user has not clicked, we can compute the probability of the user clicking on any ad P(click) = # clicks/ (# clicks + # non-clicks).  Similarly, for any Queryhash - AdDomain combination we can compute the probability of a click  ie P(click) = # clicks/ (# clicks + # non-clicks). At training time, every time we see a specific user, we can use their probability  of click as the input feature to the model. The same goes for the QueryHash - AdDomain pairs.

Suppose there were 10,000 users. One-hot encoding would generate a sparse vector of length 10,000, with a single 1 in the column that corresponds to the value of the current data point. Bin counting would encode all 10,000 binary columns with a single feature with a real value between 0 and 1. In a weird way we moved from categorical feature to real-valued feature but in this case it can actually make sense since users with 0.9 chance of clicking are of a bigger interest than users with 0.1 chance of clicking. So this is better than leaving it to the model to realize which users are more likely to click, yay, feature engineering.

We can include other features in addition to the historical click-through probability: the raw counts themselves (number of clicks and nonclicks), the log-odds ratio, or any other derivatives of probabilityy. Our example used here is for predicting ad click-through rates, but the technique readily applies to general binary classification. It can also be readily extended to multiclass classification usuing the usual techniques to extend binary classifiers to multiclass; ie via one-against-many odds ratios or other multiclass label encodings.

In short, bin counting converts a categorical variable into statistics about the value. It turns a large, sparse, binary representation of the categorical variable, such as that produced by one-hot encoding, into a very small, dense, real-valued numeric representation. (The rest of the statistics can be derived on the fly from the raw counts.) Hence it requires O(k) space, where k is the number of unique values of the categorical value.

<b><h4>Odds Ratio and Logs Odds Ratio for Bin Counting</h4></b>

The odds ratio is usually defined between two binary variables. It looks at their strength of association by asking the question, "How much more likely is it for Y to be true when X is true? For instance, we might ask, "How much more likely is a user to click on an ad than the general population?" Here X is the binary variable for the "current user  versus the rest of the population" and Y is the variable "click on add or not". The computation uses what's called the two-way contingency table (basically, four numbers that correspond to the four possible combinations of X and Y).

Given an input variable X and a target variable Y, the odds ratio is defined as:

$odds\;ratio$ = $\frac{ P(Y = 1 | X = 1) / P(Y = 0 | X = 1) }{ P(Y = 1 | X = 0)/ P( Y = 0 | X = 0)}$

More simpy, we can just look at the numerator, which examines how much more likely that a single user clicks on an ad versus not clicking. This is suitable for large categorical variables with many values, not just two: 

$odds\;ratio (user, ad click)$ = $\frac{P(Y = click | X = user)}{P(Y = not click | X = user)}$ 

In our example, this translates as the ratio between "how much more likely is it that a specific user clicks on an ad rather than does not click" and "how much more likely is it that other people click rather than not click".

Probability ratios can easily become very small or very large. For instance, there will be users who almost never click on ads, and perhaps users who click on ads much more frequently than not. The log transform again comes to our rescue. Another useful property of the logarithm is that it turns a division into a subtraction.

$log$-$odds$ $ratio (user, ad click)$  = $logP(Y = click | X = user) - log P(Y = not click | X = user)$

<b><h4>Bin-counting example</h4></b>

To illustrate bin counting in practice, we'll use data from a Kaggle competition hosted by Avazu. Here are
some relevant statistics about the dataset:

- There are 24 variables, including click, a binary click/no click counter, and device_id, which tracks which device
an ad was displayed on.

The aim of the Avazu competition was to predict click-through rate using ad data, but we will use the dataset to 
demonstrate how bin counting can greatly reduce the feature space for large amounts of streaming data.

The dataset here has 50,000 records that have a total of 8466 devices. For each device, we want to calculate the generate the bin counting statistics of number of clicks, number of no clicks, total number of ads seen, probability of clicks, probability of no clicks and the odds ratio.

To generate the number of clicks for each device id, we index all rows where the value in the clicks column, is greater than 0, then select the device id column and use the value_counts() function. The value_counts() function which will give us how many times each device clicked an ad, as a Series, with device_id as the index and the count as the value.
We create a new Series out of this, so that we get to name the Series this time as the click series. 

Next, we select all the rows where the click column is less than 1 ie 0, select the device id column and again use the value_counts() function to generate the number of times each device did not click an add, converting the result to a a Series, so that we get the chance to name each Series, called no_click. 

Then with the two Series, we create a Dataframe, the Series names become column names in the DataFrame, the device_id will be the index and we fill all NA values with 0. Finally we count the total number of times each device saw an ad by summing the clicks and no clicks columns, but before we do that we first convert each column to an int64 dtype. Now we have generated our statistics.

To bin the counts, we calculate the probability of clicks by dividing each count value by the total ads seen values, storing the result in its own column. We also calculate the no click probability by dividing the each no click value by its corresponding total ads seen value. Due to pandas vectorized implementation, we get the probabilities by dividing the click or no click column by the total ads seen column respectively but we first convert each column into dtype int64. Next, we calculate the odds ratio of clicking by dividing the click probability to the no click probability. We could use np.log10() to convert this to logarithm scale to further bin the real-valued odds-ratio into logarithm bins.
We now what we need to replace each device id, with statistics and probabilities that describe its activity.

In the implementation we use the filter() function. This function can take column / row labels as the items keyword, or it can be used to filter based on regex or even like keywords.


In [80]:
df = pd.read_csv('../datasets/50krecords.csv')

In [81]:
df.shape

(50000, 24)

In [82]:
# How many unique features for device_id categorical variable
len(df['device_id'].unique())

8466

In [103]:
# For each category, we want to calculate:
# Theta = [counts, p(click), p(no click), p(click)/p(no click)]

def click_counting(x, bin_column):
    clicks = pd.Series(x[x['click'] > 0][bin_column].value_counts(),
                       name='clicks') # clicked ads rows, for device_id get how many times user clicked
    no_clicks = pd.Series(x[x['click']< 1][bin_column].value_counts(),
                          name='no_clicks') # unclicked ads rows, for device_id, get how many times user didnt click
    counts = pd.DataFrame([clicks, no_clicks]).T.fillna('0') # device_id index, # times clicked, # times not clicked

    counts['total_clicks'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64') # total ads col

    return counts


In [104]:
def bin_counting(counts):
    counts['N+'] = counts['clicks'].astype('int64').divide(counts['total_clicks'].astype('int64')) # prob click
    counts['N-'] = counts['no_clicks'].astype('int64').divide(counts['total_clicks'].astype('int64')) # prob no_click
    counts['log_N+'] = counts['N+'].divide(counts['N-']) # odds ratio
    bin_counts = counts.filter(items=['N+', 'N-', 'log_N+']) # items=col labels ,filter also takes regex= or like=
    return counts, bin_counts

In [105]:
# Bin counts example: device_id
bin_column = 'device_id'
device_clicks = click_counting(df.filter(items=[bin_column, 'click']), bin_column)
device_all, device_bin_counts = bin_counting(device_clicks)

In [106]:
# Check to make sure we have all the devices
len(device_bin_counts)

8466

In [107]:
device_all.sort_values(by='total_clicks', ascending=False).head(4)

,clicks,no_clicks,total_clicks,N+,N-,log_N+
device_id,,,,,,
a99f214a,7143.0,34184.0,41327,0.172841,0.827159,0.208957
0f7c61dc,17.0,7.0,24,0.708333,0.291667,2.428571
c357dbff,15.0,7.0,22,0.681818,0.318182,2.142857
936e92fb,1.0,12.0,13,0.076923,0.923077,0.083333


In [100]:
device_bin_counts

,N+,N-,log_N+
device_id,,,
a99f214a,0.172841,0.827159,0.208957
0f7c61dc,0.708333,0.291667,2.428571
c357dbff,0.681818,0.318182,2.142857
3cdb4052,0.750000,0.250000,3.000000
afeffc18,0.250000,0.750000,0.333333
...,...,...,...
5303b711,0.000000,1.000000,0.000000
f1e3adcb,0.000000,1.000000,0.000000
1f201274,0.000000,1.000000,0.000000


In [101]:
df.click.min()

0

<b><h5>What about rare categories</h5></b>

Just like rare words, reare categories require special treatment. Think about a user who logs in once a year: there will be very little data to reliably estimate that user's click-through rate for ads. Moreover, rare categories waste sapce in the counts table.

One way to deal with this is through back-off, a simple technique that accumulates the counts of all rare categories in a special bin. If the (total category) count is greater than a certain threshold, the the category get its own statistics. Otherwise, we use the statistics from the back-off bin. This essentially reverts the statistics for a single rare category to the statistics computed on all rare categories. When using the back-off method, it helsps to also add a binary indicator for whether or not the statistics come from the back-off bin.

There is another way to deal with this problem, called the count-min sketch. In this method, all the categories, rare or frequent alike, are mapped through multiple hash functions with an output range, m, much smaller than the number of categories, k. When retrieving a statistic, recompute all the hashes of the category and return the smallest statistic. Having multiple hash functions mitigates the probability of collision within a single hash function. The scheme works because the number of hash functions times m, the size of the hash table can be made smaller than k, the number of categories, and still retain low overall collision probability. This way, instead of dropping the rare categories we use hashing to reduce the storage space.

<b><h5>Guarding against data leakage</h5></b>

Since bin counting relies on historical data to generate the necessary statistics, it requires waiting through a data collection period, incurring a slight delay in the learning pipeline. Also, when the data distribution changes, the counts need to be updated. The faster the data changes, the more frequently the counts need to be recomputed. This is particularly important for applications like targeted advertising, where user preferences and popular queries change very quickly, and lack of adaptation to the current distribution could mean huge losses for the advertising platform.

One might ask, why not use the same dataset to compute the relevant statistics and train the model? The big problem here is that the statistics involve the target variable, which is what the model tries to predict. Using the output to compute the input features leads to a pernicious problem known as leakage. In short, leakage means that information is revealed that gives it an unrealistic advantage to make better predictions. This could happen when test data is leaked into the training set, or when data from the future is leaked to the past. Any time that a model is given information that it shouldn't have access to when it is making predictions in real time in production, there is leakage.

If the bin-counting procedure used the current data point's label to compute part of the input statistic, that would constitute direct leakage. One way to prevent that is by instituting strict separation between count collection (for computing bin-count statistics) and training. We use an earlier batch of data points for counting, use current data points for training (mapping categorical variables to historical statistics we just collected) and use future data points for testing. That is we use time windows to prevent leakage during bin counting. This fixes the problem of leakage, but introduces the aforementioned delay (the input statistics and therefore the model will trail behind current data).

It turns out that there is another solution, based on differential privacy. A statistic is approximately leakage-proof if tis distribution stays roughly the same with or without any one data point. In practice, adding a small random noise with distribution Laplace(0,1) is sufficient to cover up any potential leakage from a single data point. This idea can be combined with leaving-one-out counting to formulate statistics on current data.

<b><h5>Counts without bounds</h5></b>

If the statistics are updated continuously given more and more historical data, the raw counts will grow without bounds. This could be a problem for the model. A trained model 'knows' the input data up to the observed scale. A trained decision tree might say, 'When x is greater than 3, predict 1'. A trained linear model might say, 'Multiply x by 0.7 and see if the result is greater than the global average'. These might be the correct decisions when x lies between 0 and 5. But what happens beyond that? No one knows.

When the input counts increase, the model will need to be retrained to adapt to the current scale. If the counts accumulate rather slowly, then the effective scale won't change too fast, and the model will not need to be retrained too frequently. But when the counts increment very quickly, frequent training will be a nuisance.

For this reason, it is often better to use normalized counts that are guaranteed to be bounded in a known interval. For instance, the estimated click-through probability is bounded between [0,1]. Another method is to take the log transform, which imposes a strict bound, but the rate of increase will be very slow when the count is very large.

Neither method will guard against shifting input distributions, eg seasonal style changes etc. The model will need to be retrained to accomodate these more fundamental changes in input data distribution, or the whole pipeline will need to move on to an online learning setting where the model is continuously adpating to the input.